# 多机存储波形
我们已经能够把历史波形保存在数据库，
但是有一个问题未考虑，多台设备同时在线，
只有一个服务器，如何区分波形是哪台设备的？

我们给波形表增加一个外键，引用设备表。
每一行通过外键指明这行波形数据来自哪台设备。
此时还未建**设备信息表**，无法引用外键。

# 机器信息
定义设备：
```python
class Device(models.Model):
    dev_id = 1
    serial = models.CharField(max_length=10)
```
设备的ID号，自增字段，作为主键。
每个设备在数据库唯一的ID。

序列号字段，若干长度的字符串。
序列号一般是出厂前写在设备内的，而且是唯一的，
通过序列号能够追踪设备。

```sql
CREATE TABLE `medical_monitor`.`device` (
  `dev_id` INT AUTO_INCREMENT,
  `serial` VARCHAR(16) UNIQUE,
  PRIMARY KEY (`dev_id`));
```

dev_id 字段自增的ID号，用PRIMARY KEY修饰为主键，
主键能够唯一的表示一行数据。

serial字段用UNIQUE修饰，数据库会保证每一行的serial值都不同。
可以尝试插入重复的serial，数据库一定会报错。

```sql
INSERT INTO `medical_monitor`.`device` (`serial`) 
VALUES ('DEV-001');
```

执行插入语句并未指明dev_id，
但是插入仍然能够成功。
```sql
SELECT * FROM device 
```
我们查看数据表，dev_id自动赋值为1，
再插入一行数据dev_id为2，
这就是AUTO_INCREMENT自增字段的作用。

# 设备登录

注意，本章代码都是在设备上运行的。
设备只在开箱时需要输入用户名和密码，之后可以一直保存在程序里面。

设备传输波形的同时需要跟着设备id参数，
以区别是哪台设备的波形。
设备在开机时连接数据库，
通过存在设备内的serial查询本机在数据库的id。
（我们自行开发的设备软件，
所以可以设置一个表单文本框，专门填写序列号；
或者通过配置文件保存serial。）

```cpp
query.prepare("SELECT * FROM device "
              "WHERE serial = :serial");
query.bindValue(":serial", "DEV-001");

int self_id = -1;
if(query.exec() && query.next())
{
    qDebug() << query.value("dev_id");
    self_id = query.value("dev_id").toInt();
}
```
每次开机时连接数据库，获得self_id，作为设备的标识。

# 多设备波形存储
修改波形表，增加设备id字段。
并且增加外键。
```sql
ALTER TABLE `medical_monitor`.`sample` 
    ADD COLUMN `dev_id` INT NULL AFTER `time`;

ALTER TABLE `medical_monitor`.`sample` 
    ADD CONSTRAINT `fk_sample_device`
      FOREIGN KEY (`dev_id`)
      REFERENCES `medical_monitor`.`device` (`dev_id`)
      ON DELETE NO ACTION  
      ON UPDATE NO ACTION;
```
解释修改表的语法：

解释外键语法：

插入波形的语句随之改变，增加device id字段的值。
```cpp
query.prepare("INSERT INTO sample(time, value, dev_id)"
              "VALUES(:time, :array, :dev_id)");
query.bindValue(":dev_id", self_id);
```

# 新建设备

系统数据库在搭建时是空的，没有设备列表。
所以我们要新建设备列表。

因为设备出厂有serial，可以在设备第一次登录系统的时候，
把serial初始化进去系统。
```cpp
query.prepare("SELECT * FROM device "
              "WHERE serial = :serial");
query.bindValue(":serial", "DEV-001");

if(query.exec() && query.size() == 0)
{
    query.prepare("INSERT INTO `medical_monitor`.`device` (`serial`) "
              "VALUES (:serial)");
    query.bindValue(":serial", "DEV-001");
    query.exec();
}
```
```query.exec()``` 代表SQL查询执行成功，
并且 ```query.size() == 0``` 说明查询的结果空，即数据库没记录此设备。
此时增加一条记录即可。

## 在线状态
因为TCP/IP基于报文交换的协议而无电路连接，
为了获取设备在线状态，用到保活机制：
- 数据库设置**最后一次刷新时间**的字段
- 设备端每隔一定周期（15秒）发送保活报文（定时器）
- 记录每次保活的时间，如果超过20秒，认为离线。

设备表增加refresh字段，代表刷新时间。

```sql
ALTER TABLE `medical_monitor`.`device` 
    ADD COLUMN `refresh` timestamp;
```

每次更新时间，注意不是新建，用到UPDATE语句。
```sql
UPDATE `medical_monitor`.`device` 
SET refresh = NOW()
WHERE dev_id =  1
```
UPDATE语法解释：

这里用到数据库自带的now函数，返回当前的时间。

看数据库表格，是否已经更新？

## 判定在线

查询到设备的刷新时间，最后一次刷新与当前的时间距离多久？

# 查看不同设备的历史波形